In [1]:
################################################################################
#
####                               IMPORTING
#
################################################################################

#\\Standard libraries
from cmath import phase
from distutils.command.config import config
import pickletools
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle as pkl
import yaml
import os
from numpy import linalg as LA
import scipy.io as sio
import random

import sys
sys.path.append(os.path.dirname(os.getcwd()))

#\\Own libraries
from model.classic_detectors import *
from data.sample_generator import *
from utils.util import *
from runners.runner_langevin_real_data import *
from runners.runner_classic_methods_real_data import *
from pathlib import Path

In [2]:
################################################################################
#
####                              SETTINGS
#
################################################################################

######################
###  Load parameters of the system ###
######################
dirPath = os.path.dirname(os.getcwd())
with open(dirPath + '/config_real_data.yml', 'r') as f:
    aux = yaml.load(f,  Loader=yaml.FullLoader)
config = dict2namespace(aux)    


######################
###  General setup ###
######################
SEED = 123
torch.manual_seed(SEED)
useGPU = False # If true, and GPU is available, use it.
LANGEVIN_DETECTOR = True
CLASSICAL_DETECTORS = True

#\\\ Determine processing unit:
if useGPU and torch.cuda.is_available():
    torch.cuda.empty_cache()
    device = 'cuda:0'
else:
    device = 'cpu'

#\\\ Total number of data frames (200 max)
int_min = 0
int_max = 200

#\\\ Index of data (allows to remove pilot signlas from array)
with open(dirPath + '/data/dataIndex', 'rb') as fp:
    dataIndex = pkl.load(fp)


In [5]:
################################################################################
#
####                               MAIN
#
################################################################################


#Create generator   
error_mmse_paper = np.zeros((48, 10))
error_lang_paper = np.zeros((48, 10))

for subcarrierIndex in range(48):
    for ofdmSymb in range(10):

        #\\\ Load data
        subcarrier = dataIndex[subcarrierIndex]
        batch_size, H_MMSE, H_comp, HPilotri, yPilotri, xri, xPilotri, pilotValue, pilotIndex, sigma, yri = loadData(ofdmSymb, subcarrier, subcarrierIndex, int_min, int_max,dirPath, dataIndex,config)

        #\\\ Create generator and indices of the true symbols
        generator = sample_generator(batch_size, config.mod_n, config.NR)
        idx_real = generator.demodulate(xri)
        j_indices = generator.joint_indices(idx_real)


        if CLASSICAL_DETECTORS == True:
            #\\\ Run MMSE detector
            serMMSE, serSDR, x_MMSE = runClassicDetectors(config, generator, batch_size, device, H = H_MMSE, y = yri, noise_sigma = sigma  , j_indices = j_indices)
            
            #\\\ Phase correction
            samplesP = torch.zeros((batch_size, config.NT * 2, 4))
            index = 0
            #\\\ Pilot evaluation
            for pilotInd in range(len(pilotIndex)):
                serLangevin, serSDR, samplesP[:,:,index] = runClassicDetectors(config, generator, batch_size, device, H = HPilotri[:,:,:,index], y = yPilotri[index,:,:], noise_sigma = sigma, j_indices = j_indices)
                index =  index + 1

            samplesP_complexMMSE = samplesP[:, 0:8, :].cpu().detach().numpy() + 1j * samplesP[:, 8:, :].cpu().detach().numpy()
            phase_corr = samplesP_complexMMSE * np.conj(pilotValue)
            phase_err = np.angle(np.mean(phase_corr, 2))

            phase_compMMSE = np.exp(-1j*phase_err)

            #\\\ Here the phase correction is applied
            x_MMSE_complex = x_MMSE[:, 0:8].cpu().detach().numpy() + 1j * x_MMSE[:, 8:].cpu().detach().numpy()
            x_MMSE = np.multiply(x_MMSE_complex, phase_compMMSE)

            aux = torch.tensor(x_MMSE)
            xr = torch.real(aux)
            xi = torch.imag(aux)

            #\\\ Final estimated symbol in the real equivalent form
            x_MMSE_hat = torch.cat((xr, xi), dim=1)

        if LANGEVIN_DETECTOR == True:

            #\\\ Phase correction
            samplesP = torch.zeros((batch_size, config.NT * 2, len(pilotIndex)))
            index = 0
            #\\\ Pilot evaluation
            for pilotInd in range(len(pilotIndex)):
                serLangevin, serSDR, samplesP[:,:,index] = runClassicDetectors(config, generator, batch_size, device, H = HPilotri[:,:,:,index], y = yPilotri[index,:,:], noise_sigma = sigma, j_indices = j_indices)
                index =  index + 1

            samplesP_complexMMSE = samplesP[:, 0:8, :].cpu().detach().numpy() + 1j * samplesP[:, 8:, :].cpu().detach().numpy()
            phase_corr = samplesP_complexMMSE * np.conj(pilotValue)
            phase_err = np.angle(np.mean(phase_corr, 2))

            phase_comp = np.exp(-1j*phase_err)

            #\\\ Multiply the channel by the phase correction before running the detection
            H = H_comp  * phase_comp[:,None,]
            H = torch.tensor(H)
            Hr = torch.real(H)
            Hi = torch.imag(H)

            h1 = torch.cat((Hr, 1. * Hi), dim=2)
            h2 = torch.cat((-1. * Hi, Hr), dim=2)
            H = torch.cat((h1, h2), dim=1)

            #\\\ Run langevin detector
            serLangevin, samples_hat, samples = runLangevin(config, generator, batch_size, device, H = H, y = yri, noise_sigma = sigma, j_indices = j_indices)

            


        error_mmse_paper[subcarrierIndex, ofdmSymb] = sym_detection_absolute(x_MMSE_hat.to(device='cpu'), j_indices, generator.real_QAM_const, generator.imag_QAM_const)
        error_lang_paper[subcarrierIndex, ofdmSymb] = sym_detection_absolute(samples_hat.to(device='cpu'), j_indices, generator.real_QAM_const, generator.imag_QAM_const)

        print(sym_detection_absolute(x_MMSE_hat.to(device='cpu'), j_indices, generator.real_QAM_const, generator.imag_QAM_const))        
        print(sym_detection_absolute(samples_hat.to(device='cpu'), j_indices, generator.real_QAM_const, generator.imag_QAM_const))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


/tmp/ipykernel_69466/3419714364.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  H = torch.tensor(H)


0.0
Trajectory: 0
Trajectory: 1
1594.0
1598.0
0.0
0.0
0.0
0.0
0.0
0.0


KeyboardInterrupt: 

In [ ]:

serLangevin = 200 * 8 - np.reshape(error_lang_paper, (-1))
serMMSE = 200 * 8 - np.reshape(error_mmse_paper, (-1))

plt.figure(1)
nLang, binsLang, patches = plt.hist(x=serLangevin, bins=20, color='blue',
                            alpha=0.5, rwidth=1, label='Langevin')
nMMSE, binsMMSE, patches = plt.hist(x=serMMSE, bins=20, color='red',
                            alpha=0.7, rwidth=1, label='MMSE')


plt.grid(True, which="both")
plt.legend(loc = 1, fontsize=15)
plt.xlabel('Errors per subcarrier per OFDM symbol', fontsize=14)
plt.ylabel('Number of channels', fontsize=14)
plt.tick_params(axis='both' , labelsize=14)
plt.tight_layout()
plt.savefig('histogram.pdf')
 
plt.show()
